In [5]:
# Import libraries
using Turing
using LinearAlgebra
using Distributions
using MultivariateStats
import MultivariateStats: reconstruct
using GaussianProcesses
using StatsBase
using Statistics
using Suppressor
using JLD2
using CSV
using DataFrames, DataFramesMeta
using SplitApplyCombine
using KernelFunctions
using MCMCChains
using PyCall
using PyPlot
using Printf
import PyCall.pyfunction

# Access the matplotlib module
matplotlib = pyimport("matplotlib")
seaborn = pyimport("seaborn")
os = pyimport("os")
pyimport("scienceplots")
np = pyimport("numpy")


PyCall.pygui(:tk)

# Set a seed for reproducibility
using Random
Random.seed!(11);

#import Pkg
#Pkg.add("PrettyTables")
FONTSIZE=20.5;

In [6]:
matplotlib_backends_pdf = pyimport("matplotlib.backends.backend_pdf")
PdfPages = matplotlib_backends_pdf.PdfPages

PyObject <class 'matplotlib.backends.backend_pdf.PdfPages'>

In [7]:
#Loading all calibrated posteriors from a given realization of future observations, storing them into a dictionary

#Note that this function assumes a specific file structure of the HPC_Output_Folder:
    # Example:   Root/output_R1_job808_task0/chains_dir/2030_chain.jld2

function load_data(realization, HPC_Results_Folder, saveout_directory)
    post_data = Dict{Int64, Matrix{Float64}}()

    for dir in readdir(HPC_Results_Folder)
        if occursin("_R$(realization)_", dir)
            chain_path = joinpath(HPC_Results_Folder, dir, "chains_dir" )
            #Read only the chains, not the generated quantities, for now
            chain_string = only(readdir(chain_path))
            #Get the current year
            year = parse(Int, SubString(chain_string, 1:4))
            #Load the chain from disc
            current_chain = JLD2.load("$(chain_path)/$(chain_string)", "chain" )
            #Convert the chain into a matrix
            chain_matrix = get_params(current_chain)
            θ_post = [chain_matrix.vmThresh;; chain_matrix.fricExp;; chain_matrix.mu_scale;; 
                         chain_matrix.stiff_scale;; chain_matrix.gamma0;; chain_matrix.melt_flux]
            #Add this matrix to the dicrtionary according to its last year of constraining observation
            post_data[ year ] = θ_post 
    
        end
    end
    #Save this dictionary to disk
    @save "$(saveout_directory)/R_$(realization)_Posterior_Dict.jld2" post_data
    
    return nothing
end


load_data (generic function with 1 method)

In [8]:
#Define the path to where your HPC results are
hpc_data_directory = "../Data/HPC_Data" # "../../BrookhavenCode/ProjectNotebooks/MCMC_Outputs_Sanket"
saveout_location = "../Data/Posterior_Data"

#Realization numbers
Realizations = [string(i) for i in 1:100];


for r in Realizations
    load_data(r, hpc_data_directory , saveout_location)
end